In [8]:
import sympy as sp
sp.init_printing()

In [63]:
# Source kinematics
px, py, pz = sp.symbols('px, py, pz')
vx, vy, vz = sp.symbols('vx, vy, vz')
ax, ay, az = sp.symbols('ax, ay, az')

# speed of sound
sos = sp.symbols('sos')

# Source time, used for the source side of the estimation (before convergence)
st = sp.symbols('st')

# Apply source kinematics
def source_pos(t):
    sx = px + vx * t + 0.5 * ax * t**2
    sy = py + vy * t + 0.5 * ay * t**2
    sz = pz + vz * t + 0.5 * az * t**2
    return sx, sy, sz

# Receiver location
#rx, ry, rz = sp.symbols('rx, ry, rz')

# Receiver time, at which we're actually sampling
rt = sp.symbols('rt')

In [64]:
# Single Newton-Raphson iteration

# Initial dt estimate
dt = sp.symbols('dt')
# Estimate position with the initial time estimate
(sx, sy, sz) = source_pos(rt - dt)
# Distance between estimated source and receiver
d = sp.sqrt((sx)**2 + (sy)**2 + (sz)**2)

# This is the distance error
err = (dt * sos) - d

# The derivative of the distance error with respect to dt
err_dt = sp.diff(err, dt)

# Newton-Raphson iteration
dt2 = dt - err / err_dt

print(sp.simplify(err))
print(sp.simplify(dt2))


nr_lambda = sp.lambdify((dt, px, py, pz, vx, vy, vz, ax, ay, az, rt, sos), dt2, 'numpy')
err_lambda = sp.lambdify((dt, px, py, pz, vx, vy, vz, ax, ay, az, rt, sos), err, 'numpy')



dt*sos - sqrt((0.5*ax*(dt - rt)**2 + px - vx*(dt - rt))**2 + (0.5*ay*(dt - rt)**2 + py - vy*(dt - rt))**2 + (0.5*az*(dt - rt)**2 + pz - vz*(dt - rt))**2)
dt - (dt*sos - sqrt((0.5*ax*(dt - rt)**2 + px - vx*(dt - rt))**2 + (0.5*ay*(dt - rt)**2 + py - vy*(dt - rt))**2 + (0.5*az*(dt - rt)**2 + pz - vz*(dt - rt))**2))/(sos - ((ax*(2*dt - 2*rt) - 2*vx)*(0.5*ax*(dt - rt)**2 + px - vx*(dt - rt))/2 + (ay*(2*dt - 2*rt) - 2*vy)*(0.5*ay*(dt - rt)**2 + py - vy*(dt - rt))/2 + (az*(2*dt - 2*rt) - 2*vz)*(0.5*az*(dt - rt)**2 + pz - vz*(dt - rt))/2)/sqrt((0.5*ax*(dt - rt)**2 + px - vx*(dt - rt))**2 + (0.5*ay*(dt - rt)**2 + py - vy*(dt - rt))**2 + (0.5*az*(dt - rt)**2 + pz - vz*(dt - rt))**2))


In [65]:
sp.rust_code(dt2)

'dt - (dt*sos - ((0.5*ax*(-dt + rt).powi(2) + px + vx*(-dt + rt)).powi(2) + (0.5*ay*(-dt + rt).powi(2) + py + vy*(-dt + rt)).powi(2) + (0.5*az*(-dt + rt).powi(2) + pz + vz*(-dt + rt)).powi(2)).sqrt())/(sos - ((1_f64/2.0)*(ax*(2*dt - 2*rt) - 2*vx)*(0.5*ax*(-dt + rt).powi(2) + px + vx*(-dt + rt)) + (1_f64/2.0)*(ay*(2*dt - 2*rt) - 2*vy)*(0.5*ay*(-dt + rt).powi(2) + py + vy*(-dt + rt)) + (1_f64/2.0)*(az*(2*dt - 2*rt) - 2*vz)*(0.5*az*(-dt + rt).powi(2) + pz + vz*(-dt + rt)))/((0.5*ax*(-dt + rt).powi(2) + px + vx*(-dt + rt)).powi(2) + (0.5*ay*(-dt + rt).powi(2) + py + vy*(-dt + rt)).powi(2) + (0.5*az*(-dt + rt).powi(2) + pz + vz*(-dt + rt)).powi(2)).sqrt())'

In [71]:
sp.pycode(dt2)

'dt - (dt*sos - math.sqrt((0.5*ax*(-dt + rt)**2 + px + vx*(-dt + rt))**2 + (0.5*ay*(-dt + rt)**2 + py + vy*(-dt + rt))**2 + (0.5*az*(-dt + rt)**2 + pz + vz*(-dt + rt))**2))/(sos - ((1/2)*(ax*(2*dt - 2*rt) - 2*vx)*(0.5*ax*(-dt + rt)**2 + px + vx*(-dt + rt)) + (1/2)*(ay*(2*dt - 2*rt) - 2*vy)*(0.5*ay*(-dt + rt)**2 + py + vy*(-dt + rt)) + (1/2)*(az*(2*dt - 2*rt) - 2*vz)*(0.5*az*(-dt + rt)**2 + pz + vz*(-dt + rt)))/math.sqrt((0.5*ax*(-dt + rt)**2 + px + vx*(-dt + rt))**2 + (0.5*ay*(-dt + rt)**2 + py + vy*(-dt + rt))**2 + (0.5*az*(-dt + rt)**2 + pz + vz*(-dt + rt))**2))'

In [74]:
# Initial estimation

px_ = -20
py_ = 1
pz_ = 0
vx_ = 90
vy_ = 0
vz_ = 0
ax_ = 0
ay_ = 0
az_ = 9
rt_ = 0
sos_ = 343

import math
def nr_py(dt, px, py, pz, vx, vy, vz, ax, ay, az, rt, sos):
    return dt - (dt*sos - math.sqrt((0.5*ax*(-dt + rt)**2 + px + vx*(-dt + rt))**2 + (0.5*ay*(-dt + rt)**2 + py + vy*(-dt + rt))**2 + (0.5*az*(-dt + rt)**2 + pz + vz*(-dt + rt))**2))/(sos - ((1/2)*(ax*(2*dt - 2*rt) - 2*vx)*(0.5*ax*(-dt + rt)**2 + px + vx*(-dt + rt)) + (1/2)*(ay*(2*dt - 2*rt) - 2*vy)*(0.5*ay*(-dt + rt)**2 + py + vy*(-dt + rt)) + (1/2)*(az*(2*dt - 2*rt) - 2*vz)*(0.5*az*(-dt + rt)**2 + pz + vz*(-dt + rt)))/math.sqrt((0.5*ax*(-dt + rt)**2 + px + vx*(-dt + rt))**2 + (0.5*ay*(-dt + rt)**2 + py + vy*(-dt + rt))**2 + (0.5*az*(-dt + rt)**2 + pz + vz*(-dt + rt))**2))


last_dt = 0
errs = []
dts = []
for i in range(10):
    errs.append(err_lambda(last_dt, px_, py_, pz_, vx_, vy_, vz_, ax_, ay_, az_, rt_, sos_))
    new_dt = nr_py(last_dt, px_, py_, pz_, vx_, vy_, vz_, ax_, ay_, az_, rt_, sos_)
    dts.append(new_dt)
    last_dt = new_dt


for err_, dt_ in zip(errs, dts):
    print(f'{err_ * 44100 / sos_}, {dt_}')



-2574.64085072153, 0.07911502216656353
-0.30138234576462253, 0.07912428511186843
-2.364280200189179e-09, 0.0791242851119411
-4.567774729886359e-13, 0.07912428511194111
-4.567774729886359e-13, 0.07912428511194113
0.0, 0.07912428511194113
0.0, 0.07912428511194113
0.0, 0.07912428511194113
0.0, 0.07912428511194113
0.0, 0.07912428511194113


In [69]:
# Incremental update
px_ = 0
py_ = 1
pz_ = 0
vx_ = 0
vy_ = 0
vz_ = 0
ax_ = 0
ay_ = 0
az_ = 0
rt_ = 0
sos_ = 343

rvx = 900
rvy = 0
rvz = 0
rax = 0
ray = 0
raz = 0

# Initial estimate
ldt = 0

for i in range(10):
    ldt = nr_lambda(ldt, px_, py_, pz_, vx_, vy_, vz_, ax_, ay_, az_, rt_, sos_)

for sample in range(1024):
    tadv = sample / 44100
    pxn = px_ - (rvx * tadv + 0.5 * rax * tadv**2)
    pyn = py_ - (rvy * tadv + 0.5 * ray * tadv**2)
    pzn = pz_ - (rvz * tadv + 0.5 * raz * tadv**2)


    sdt = ldt
    serr = err_lambda(sdt, pxn, pyn, pzn, vx_, vy_, vz_, ax_, ay_, az_, rt_, sos_)
    ldt = nr_lambda(sdt, pxn, pyn, pzn, vx_, vy_, vz_, ax_, ay_, az_, rt_, sos_)
    nerr = err_lambda(ldt, pxn, pyn, pzn, vx_, vy_, vz_, ax_, ay_, az_, rt_, sos_)

    print(f'{serr * 44100 / sos_}, {ldt}, {nerr * 44100 / sos_}')
    

0.0, 0.0029154518950437317, 0.0
-0.026771770945321904, 0.00291605896467968, 0.0
-0.08028189337311151, 0.002917879415776576, 0.0
-0.1336919656549616, 0.002920910979623627, 0.0
-0.1869359788157578, 0.0029251498907078848, 0.0
-0.2399489500712036, 0.002930590909983876, 0.0
-0.2926673126001208, 0.002937227356981611, 0.0
-0.3450292852523422, 0.002945051150297991, 0.0
-0.3969752173486147, 0.002954052855906803, 0.0
-0.44844790434132503, 0.002964221742626561, 0.0
-0.4993928708099559, 0.0029755458440054714, 0.0
-0.5497586180263113, 0.002988012025820127, 0.0
-0.5994968341195156, 0.003001606058339844, 0.0
-0.6485625656725803, 0.0030163126924820792, 0.0
-0.696914350366156, 0.003032115738975643, 0.0
-0.7445143110232763, 0.003048998149656443, 0.0
-0.791328212091234, 0.003066942100043999, 0.0
-0.8373254801989661, 0.0030859290723841116, 0.0
-0.882479190943708, 0.0031059399383919052, 0.0
-0.9267660244834022, 0.003126955040987901, 0.0
-0.9701661928337689, 0.003148954274385492, 0.0
-1.0126633419981086, 0.